In [364]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings('always') 
warnings.filterwarnings('ignore') 

import matplotlib.pyplot as plt
import seaborn as sns
import random as rnd

# %matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier


# Performing ETL